You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from two specific endpoints: 
	All launches: https://api.spacexdata.com/v3/launches
	All rockets: https://api.spacexdata.com/v3/rockets
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	JOIN Operation:
o	Perform a "JOIN" operation to combine information from the launches data and the rockets data.
o	Join Logic: For each launch record, look up the corresponding rocket record using the rocket identifier. The join key is the rocket.rocket_id field within each launch record, which matches the rocket_id field in the rocket records. Add the rocket_name from the matched rocket record to the launch record. (This is conceptually similar to a left join, keeping all launch records).
o	The final result should be a list of dictionaries, where each dictionary represents a launch enriched with the corresponding rocket_name.
3.	Control Parameters and Debugging:
o	Include variables at the beginning of the script to define the API endpoint URLs, making them easily modifiable: 
	LAUNCHES_API_URL = "https://api.spacexdata.com/v3/launches"
	ROCKETS_API_URL = "https://api.spacexdata.com/v3/rockets"
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval (for launches, for rockets), number of launches retrieved, number of rockets retrieved, starting join operation, join operation complete, number of records after join, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + join operation). Print this time after the join operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting joined list of dictionaries from the join operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting joined JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.


In [0]:
import requests
import json
import logging
import time

# Define API endpoint URLs
LAUNCHES_API_URL = "https://api.spacexdata.com/v3/launches"
ROCKETS_API_URL = "https://api.spacexdata.com/v3/rockets"

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Record start time of the entire script
start_time = time.time()

def retrieve_data(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logger.error(f"Error retrieving data from {url}: {e}")
        return []

def join_launches_with_rockets(launches, rockets):
    rocket_dict = {rocket['rocket_id']: rocket['rocket_name'] for rocket in rockets}
    joined_launches = []
    for launch in launches:
        rocket_id = launch.get('rocket', {}).get('rocket_id')
        if rocket_id in rocket_dict:
            launch['rocket_name'] = rocket_dict[rocket_id]
        joined_launches.append(launch)
    return joined_launches

def upload_data(data):
    try:
        response = requests.post('https://httpbin.org/post', json=data)
        response.raise_for_status()
        logger.info(f"Upload outcome: {response.json()['data']}")
    except requests.exceptions.RequestException as e:
        logger.error(f"Error uploading data: {e}")

def main():
    # Retrieve launches data
    logger.info("Starting data retrieval for launches")
    launches = retrieve_data(LAUNCHES_API_URL)
    logger.info(f"Number of launches retrieved: {len(launches)}")

    # Retrieve rockets data
    logger.info("Starting data retrieval for rockets")
    rockets = retrieve_data(ROCKETS_API_URL)
    logger.info(f"Number of rockets retrieved: {len(rockets)}")

    # Record start time of the main operations
    main_start_time = time.time()

    # Perform join operation
    logger.info("Starting join operation")
    joined_launches = join_launches_with_rockets(launches, rockets)
    logger.info("Join operation complete")
    logger.info(f"Number of records after join: {len(joined_launches)}")

    # Record end time of the main operations
    main_end_time = time.time()
    logger.info(f"Time taken for main operations: {main_end_time - main_start_time} seconds")

    # Upload joined data
    logger.info("Starting upload to httpbin")
    upload_data(joined_launches)

# Run the main function
main()

# Record end time of the entire script
end_time = time.time()
logger.info(f"Total execution time: {end_time - start_time} seconds")
